<a href="https://colab.research.google.com/github/tinkvu/ZomatoRecommender/blob/main/Restaurant_Recommendation_%7C_Zomato_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Content Based Filtering | Zomato Dataset

<B> Recommending Restaurants based on Reviews </b>

In [ ]:
#importing necessary libraries
from google.colab import drive
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import string
from nltk.corpus import stopwords

In [ ]:
#importing dataset
drive.mount('/content/drive')
csv_file_path = '/content/drive/My Drive/Zomato Recommender/Dataset/zomato.csv'
df = pd.read_csv(csv_file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [ ]:
print("shape: ", df.shape)
print("\n \n Columns: ",df.columns)
df.info()

shape:  (51717, 17)

 
 Columns:  Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost(for two people)', 'reviews_list', 'menu_item',
       'listed_in(type)', 'listed_in(city)'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   

#Data Preprocessing

In [ ]:
#Deleting Unnnecessary Columns
zomato=df.drop(['url','dish_liked','phone'],axis=1) #Dropping unnecessary columns
zomato

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51712,"Four Points by Sheraton Bengaluru, 43/3, White...",Best Brews - Four Points by Sheraton Bengaluru...,No,No,3.6 /5,27,Whitefield,Bar,Continental,"1,500","[('Rated 5.0', ""RATED\n Food and service are ...",[],Pubs and bars,Whitefield
51713,"Number 10, Garudachar Palya, Mahadevapura, Whi...",Vinod Bar And Restaurant,No,No,NaN,0,Whitefield,Bar,Finger Food,600,[],[],Pubs and bars,Whitefield
51714,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Plunge - Sheraton Grand Bengaluru Whitefield H...,No,No,NaN,0,Whitefield,Bar,Finger Food,"2,000",[],[],Pubs and bars,Whitefield
51715,Sheraton Grand Bengaluru Whitefield Hotel & Co...,Chime - Sheraton Grand Bengaluru Whitefield Ho...,No,Yes,4.3 /5,236,"ITPL Main Road, Whitefield",Bar,Finger Food,"2,500","[('Rated 4.0', 'RATED\n Nice and friendly pla...",[],Pubs and bars,Whitefield


In [ ]:
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)

In [ ]:
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True) #Removes any row with NaN in any fields
zomato.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   address                      43499 non-null  object
 1   name                         43499 non-null  object
 2   online_order                 43499 non-null  object
 3   book_table                   43499 non-null  object
 4   rate                         43499 non-null  object
 5   votes                        43499 non-null  int64 
 6   location                     43499 non-null  object
 7   rest_type                    43499 non-null  object
 8   cuisines                     43499 non-null  object
 9   approx_cost(for two people)  43499 non-null  object
 10  reviews_list                 43499 non-null  object
 11  menu_item                    43499 non-null  object
 12  listed_in(type)              43499 non-null  object
 13  listed_in(city)              43499 n

In [ ]:
#Reading Column Names
print("Before: ",zomato.columns)

#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
print("Updated :",zomato.columns)

Before:  Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'approx_cost(for two people)',
       'reviews_list', 'menu_item', 'listed_in(type)', 'listed_in(city)'],
      dtype='object')
Updated : Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city'],
      dtype='object')


In [ ]:
print(zomato['cost'])
#Some Transformations
zomato['cost'] = zomato['cost'].str.replace(',', '')  # Remove commas
zomato['cost'] = pd.to_numeric(zomato['cost'])  # Convert to numeric

# Check the result
print(zomato['cost'])

zomato.info()

0          800
1          800
2          800
3          300
4          600
         ...  
51709      800
51711      800
51712    1,500
51715    2,500
51716    1,500
Name: cost, Length: 43499, dtype: object
0         800
1         800
2         800
3         300
4         600
         ... 
51709     800
51711     800
51712    1500
51715    2500
51716    1500
Name: cost, Length: 43499, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 43499 entries, 0 to 51716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   address       43499 non-null  object
 1   name          43499 non-null  object
 2   online_order  43499 non-null  object
 3   book_table    43499 non-null  object
 4   rate          43499 non-null  object
 5   votes         43499 non-null  int64 
 6   location      43499 non-null  object
 7   rest_type     43499 non-null  object
 8   cuisines      43499 non-null  object
 9   cost          43499 non-null 

In [ ]:
#Reading Rate of dataset
print("Before: ",zomato['rate'].unique())

# Assuming zomato is your DataFrame and removing /5 from each rows
zomato['rate'] = zomato['rate'].str.replace('/5', '')

# Check the result
print("After: ",zomato['rate'].unique())

Before:  ['4.1/5' '3.8/5' '3.7/5' '3.6/5' '4.6/5' '4.0/5' '4.2/5' '3.9/5' '3.1/5'
 '3.0/5' '3.2/5' '3.3/5' '2.8/5' '4.4/5' '4.3/5' 'NEW' '2.9/5' '3.5/5'
 '2.6/5' '3.8 /5' '3.4/5' '4.5/5' '2.5/5' '2.7/5' '4.7/5' '2.4/5' '2.2/5'
 '2.3/5' '3.4 /5' '-' '3.6 /5' '4.8/5' '3.9 /5' '4.2 /5' '4.0 /5' '4.1 /5'
 '3.7 /5' '3.1 /5' '2.9 /5' '3.3 /5' '2.8 /5' '3.5 /5' '2.7 /5' '2.5 /5'
 '3.2 /5' '2.6 /5' '4.5 /5' '4.3 /5' '4.4 /5' '4.9/5' '2.1/5' '2.0/5'
 '1.8/5' '4.6 /5' '4.9 /5' '3.0 /5' '4.8 /5' '2.3 /5' '4.7 /5' '2.4 /5'
 '2.1 /5' '2.2 /5' '2.0 /5' '1.8 /5']
After:  ['4.1' '3.8' '3.7' '3.6' '4.6' '4.0' '4.2' '3.9' '3.1' '3.0' '3.2' '3.3'
 '2.8' '4.4' '4.3' 'NEW' '2.9' '3.5' '2.6' '3.8 ' '3.4' '4.5' '2.5' '2.7'
 '4.7' '2.4' '2.2' '2.3' '3.4 ' '-' '3.6 ' '4.8' '3.9 ' '4.2 ' '4.0 '
 '4.1 ' '3.7 ' '3.1 ' '2.9 ' '3.3 ' '2.8 ' '3.5 ' '2.7 ' '2.5 ' '3.2 '
 '2.6 ' '4.5 ' '4.3 ' '4.4 ' '4.9' '2.1' '2.0' '1.8' '4.6 ' '4.9 ' '3.0 '
 '4.8 ' '2.3 ' '4.7 ' '2.4 ' '2.1 ' '2.2 ' '2.0 ' '1.8 ']


In [ ]:
#Removing "NEW"valued rows
zomato = zomato[zomato['rate'] != 'NEW']
zomato = zomato[zomato['rate'] != '-']
# Check the result
print(zomato['rate'].unique())


['4.1' '3.8' '3.7' '3.6' '4.6' '4.0' '4.2' '3.9' '3.1' '3.0' '3.2' '3.3'
 '2.8' '4.4' '4.3' '2.9' '3.5' '2.6' '3.8 ' '3.4' '4.5' '2.5' '2.7' '4.7'
 '2.4' '2.2' '2.3' '3.4 ' '3.6 ' '4.8' '3.9 ' '4.2 ' '4.0 ' '4.1 ' '3.7 '
 '3.1 ' '2.9 ' '3.3 ' '2.8 ' '3.5 ' '2.7 ' '2.5 ' '3.2 ' '2.6 ' '4.5 '
 '4.3 ' '4.4 ' '4.9' '2.1' '2.0' '1.8' '4.6 ' '4.9 ' '3.0 ' '4.8 ' '2.3 '
 '4.7 ' '2.4 ' '2.1 ' '2.2 ' '2.0 ' '1.8 ']


In [ ]:
#Capitalizing first letter of Restaurant name
zomato.name = zomato.name.apply(lambda x:x.title())

#Converting categorical values to Boolean
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)

zomato.cost.unique()

array([ 800,  300,  600,  700,  550,  500,  450,  650,  400,  900,  200,
        750,  150,  850,  100, 1200,  350,  250,  950, 1000, 1500, 1300,
        199, 1100, 1600,  230,  130, 1700, 1350, 2200, 1400, 2000, 1800,
       1900,  180,  330, 2500, 2100, 3000, 2800, 3400,   50,   40, 1250,
       3500, 4000, 2400, 2600, 1450,   70, 3200,  240, 6000, 1050, 2300,
       4100,  120, 5000, 3700, 1650, 2700, 4500,   80])

In [ ]:
zomato['city'].unique()

array(['Banashankari', 'Bannerghatta Road', 'Basavanagudi', 'Bellandur',
       'Brigade Road', 'Brookefield', 'BTM', 'Church Street',
       'Electronic City', 'Frazer Town', 'HSR', 'Indiranagar',
       'Jayanagar', 'JP Nagar', 'Kalyan Nagar', 'Kammanahalli',
       'Koramangala 4th Block', 'Koramangala 5th Block',
       'Koramangala 6th Block', 'Koramangala 7th Block', 'Lavelle Road',
       'Malleshwaram', 'Marathahalli', 'MG Road', 'New BEL Road',
       'Old Airport Road', 'Rajajinagar', 'Residency Road',
       'Sarjapur Road', 'Whitefield'], dtype=object)

In [ ]:
zomato.isnull().sum()

address         0
name            0
online_order    0
book_table      0
rate            0
votes           0
location        0
rest_type       0
cuisines        0
cost            0
reviews_list    0
menu_item       0
type            0
city            0
dtype: int64

In [ ]:
zomato['votes'].unique()

array([ 775,  787,  918, ..., 4957, 2382,  843])

In [ ]:
zomato['type'].unique()

array(['Buffet', 'Cafes', 'Delivery', 'Desserts', 'Dine-out',
       'Drinks & nightlife', 'Pubs and bars'], dtype=object)

In [ ]:
zomato.shape

(41237, 14)

In [ ]:
# Clean 'rate' column and convert to numeric
zomato['rate'] = zomato['rate'].str.extract('(\d+\.\d+)').astype(float)  # Extract and convert to float

# Computing Mean Rating
restaurants = zomato['name'].unique()
zomato['Mean Rating'] = zomato.groupby('name')['rate'].transform('mean')

In [ ]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000


In [ ]:
scaler = MinMaxScaler(feature_range = (1,5))

zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

zomato.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
34982,"5th Main, Near Muni Reddy Kalyana Mantap, Kora...",Sree Krishna Kafe,True,False,4.2,973,Koramangala 5th Block,Quick Bites,South Indian,200,"[('Rated 4.0', 'RATED\n Ambience - 3/5 Foo...","['Special Meal', 'Vada', 'Sambar Vada', 'Mini ...",Delivery,Koramangala 7th Block,3.89
31519,"307/1, 1st Main Cross, Next to Essar Petrol Bu...",Paratha Darbar,True,False,3.5,15,Koramangala 1st Block,Quick Bites,North Indian,400,"[('Rated 4.0', ""RATED\n Amazing food at affor...",[],Dine-out,Koramangala 5th Block,3.19
42009,"601, 2nd Floor, 1st Main, C Block, AECS Layout...",Kaayal,True,False,3.2,270,Brookefield,Casual Dining,"Kerala, Chinese",500,"[('Rated 3.0', 'RATED\n Kaayal is our neighbo...",[],Dine-out,Marathahalli,2.75
30548,"Hermain Complex, 16th Main, Stage 2, BTM, Bang...",Hungry Lee,False,False,4.1,357,BTM,Quick Bites,"Chinese, Thai, Asian",500,"[('Rated 4.0', 'RATED\n Tiny but terrific. Re...",[],Delivery,Koramangala 5th Block,3.97
40620,"V3/1, NGEF, Industrial Estate, Mahadevapura Po...",Biryani Factory,True,False,2.9,14,Whitefield,Delivery,Biryani,400,"[('Rated 1.0', 'RATED\n Worst biryani i had i...","['Crispy Chicken', 'Mixed Veg Biryani', 'Panee...",Delivery,Marathahalli,2.46


In [ ]:
zomato.to_csv('zomatov1.csv')

##Text Preprocessing

Here are some typical text preprocessing/cleaning procedures:

* Converting to lowercase

* Eliminating punctuation marks

* Omitting stopwords

* Deleting URLs

* Correcting spelling errors

In [ ]:
#zomato=pd.read_csv('/content/zomatov1.csv')
#zomato = zomato.drop(zomato.columns[0], axis=1)
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,True,True,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.99
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,True,False,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.97
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,True,False,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.58
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,False,False,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.45
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,False,False,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.58


In [ ]:
# 5 examples of these columns before text processing:
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
6867,"[('Rated 4.0', ""RATED\n When you crave for pi...",Pizza
25988,"[('Rated 3.0', 'RATED\n It was an first visit...","Arabian, Chinese, North Indian"
36057,"[('Rated 3.0', ""RATED\n Located on main road ...","North Indian, Biryani, Chinese"
12746,"[('Rated 5.0', ""RATED\n Trees all around, old...",Cafe
9835,"[('Rated 5.0', ""RATED\n Amazing food. Even fo...",Korean


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Lowercasing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

# Removal of Punctuation
PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):

    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

# Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):

    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

# Removal of URLs
def remove_urls(text):

    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))



# Display a sample of the preprocessed data
zomato[['reviews_list', 'cuisines']].sample(5)


,reviews_list,cuisines
8112,rated 10 ratedn dal completely burnt onions bi...,"North Indian, Fast Food"
41171,rated 20 ratedn chai galli brookfield bangalor...,"Cafe, Street Food"
11320,rated 50 ratedn delicious food reasonable pric...,North Indian
51691,rated 40 ratedn lunch ordered long island ice ...,"American, Italian"
15543,rated 25 ratedn small place quick lunch opposi...,North Indian


In [ ]:
zomato.loc[26335, 'reviews_list']

'rated 50 ratedn tried thier pudina chapati kind amazing unique never found resturants serve variety sea foods seasonally available must try dish kerela meals chicken items fried rice quite good tasty budget food amazing taste resturant little compact recommended rated 10 ratedn food nasty tasted old difficult decipher exactly food plate terrible taste follow serious food hygiene regime avoid place simple rated 20 ratedn ordered first timeit new food tasted good next time ordered chicken pieces cooked properly tasted awful atill try one time conclude restaurant'

In [ ]:
zomato.sample(5)

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
39724,"18/2, Temple Road, 10th Cross, Malleshwaram, B...",Mumbai Spice,True,False,3.7,100,Malleshwaram,Quick Bites,"North Indian, Chinese",400,rated 30 ratedn ordered meals taste ok phulka ...,"['Aloo Paratha', 'Gobi Manchurian', 'Veg Fried...",Delivery,Malleshwaram,3.45
14663,"11, 14th Cross, 2nd Block, RT Nagar, Bangalore",Mother'S Kitchen,True,False,3.7,154,RT Nagar,Casual Dining,North Indian,600,rated 10 ratedn think need raise standard inst...,[],Delivery,Frazer Town,3.45
43873,"Mezzanine Level, The Pride Hotel, 93, Richmond...",Fuel Resto Bar,False,False,4.2,38,Richmond Road,Bar,"Finger Food, Mediterranean, North Indian",2000,rated 40 ratedn good small cozy place ãx83ãx...,[],Drinks & nightlife,MG Road,4.10
35101,"Swagath Main Road, Near Garuda Mall, Jayanagar...",Pani Kum Chai,True,False,3.9,67,Jayanagar,Quick Bites,Fast Food,200,rated 30 ratedn hi hope tasted chai go ahead g...,"['Chicken Tikka', 'Chicken Kadai Gosh', 'Chick...",Delivery,Koramangala 7th Block,3.71
1729,"24th Main Road, 18th Cross, 5th Phase, JP Naga...",Kakal Kai Ruchi,False,False,3.9,30,JP Nagar,Casual Dining,"South Indian, North Indian, Chinese",450,rated 40 ratedn great taste new dishes introdu...,[],Delivery,Bannerghatta Road,3.71


In [ ]:
list(zomato['name'].unique())

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point'

In [ ]:
zomato['name'] = zomato['name'].replace('Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â© Down The Alley', 'Cafe Down the Alley')
zomato['name'] = zomato['name'].replace('Bohra Bohra Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â©', 'Bohra Bohra Thaal')

In [ ]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
restaurant_names

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village',
 'Timepass Dinner',
 'Rosewood International Hotel - Bar & Restaurant',
 'Onesta',
 'Penthouse Cafe',
 'Smacznego',
 'Cafe Down the Alley',
 'Cafe Shuffle',
 'The Coffee Shack',
 'Caf-Eleven',
 'Cafe Vivacity',
 'Catch-Up-Ino',
 "Kirthi'S Biryani",
 'T3H Cafe',
 '360 Atoms Restaurant And Cafe',
 'The Vintage Cafe',
 'Woodee Pizza',
 'Cafe Coffee Day',
 'My Tea House',
 'Hide Out Cafe',
 'Cafe Nova',
 'Coffee Tindi',
 'Sea Green Cafe',
 'Cuppa',
 "Srinathji'S Cafe",
 'Redberrys',
 'Foodiction',
 'Sweet Truth',
 'Ovenstory Pizza',
 'Faasos',
 'Behrouz Biryani',
 'Fast And Fresh',
 'Szechuan Dragon',
 'Empire Restaurant',
 'Maruthi Davangere Benne Dosa',
 'Chaatimes',
 'Havyaka Mess',
 "Mcdonald'S",
 "Domino'S Pizza",
 'Hotboxit',
 'Kitchen Garden',
 'Recipe',
 'Beijing Bites',
 'Tasty Bytes',
 'Petoo',
 'Shree Cool Point',
 'Corner House Ice Cream',
 'Biryanis And More',
 'Roving Feast',
 'Freshm

In [ ]:
print(zomato.shape)
zomato.sample(5)


(41237, 15)


,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
7609,"610, 60 Feet Main Road, C-Block, AECS Layout, ...",Apoorva Multi Cuisine Restaurant,True,False,3.9,236,Brookefield,Casual Dining,"North Indian, South Indian, Chinese, Andhra, B...",750,rated 40 ratedn around brookfield felt place b...,"['Chicken Manchow Soup', 'Tandoori Chicken', '...",Delivery,Brookefield,3.71
36255,"Vaishnavi Summit, 6/B, Summit, 80 Feet Main Ro...",Jadoo Ka Dabba,False,False,3.6,20,Koramangala 3rd Block,Delivery,"North Indian, Chinese",200,rated 50 ratedn amazing services never problem...,[],Delivery,Koramangala 7th Block,3.32
23321,"3/1, VAV Avenue, Eshwara Layout, RBI Layout Ma...",Tandoor Chai House,True,False,3.5,7,JP Nagar,Quick Bites,"Street Food, Sandwich, Rolls, Beverages",200,rated 20 ratedn biggest disappointment name ta...,"['Oreo Milkshake', 'Chocolate Milkshake', 'Cad...",Dine-out,JP Nagar,3.19
51149,"289, Opposite Whitefield Bus Stop, Near Jamia ...",Sri Lakshmi Vaibhav,True,False,4.0,160,Whitefield,Casual Dining,"South Indian, North Indian, Chinese",500,rated 20 ratedn impressed cuisine place rava o...,"['2 Idli with Vada', 'Puri', 'Rice Bath', 'Mas...",Dine-out,Whitefield,3.65
22460,"62,7th Cross, 24th Main, Phase 2, JP Nagar, be...",Apsara Ice Cream,False,False,4.3,251,JP Nagar,Dessert Parlor,"Ice Cream, Desserts",200,rated 50 ratedn pani puri pathaka inventionnnt...,[],Delivery,JP Nagar,3.88


In [ ]:
zomato.columns

Index(['address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list',
       'menu_item', 'type', 'city', 'Mean Rating'],
      dtype='object')

In [ ]:
zomato=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)

In [ ]:
zomato.sample(5)

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
1273,Waffle Stories,True,False,4.0,BTM,"Desserts, Beverages",300,rated 40 ratedn loved place desserts presentat...,Bannerghatta Road,3.73
3793,Cafe Thulp,True,True,4.0,Bellandur,"Cafe, Burger, Italian, Salad",1000,rated 30 ratedn heard koramangla branch good s...,Bellandur,3.76
5244,Mama Mia!,True,False,4.3,Indiranagar,"Desserts, Cafe",400,rated 50 ratedn mama mia 2 mins away hotel vis...,Brigade Road,4.04
26654,Sardarji Londonwaley,True,False,4.1,Koramangala 1st Block,North Indian,450,rated 50 ratedn located koramangla sardarji lo...,Koramangala 4th Block,3.90
46110,Tippler,False,True,4.2,Indiranagar,"Russian, Continental, North Indian",1300,rated 20 ratedn ugh like place crowd aged felt...,Old Airport Road,4.10


In [ ]:
zomato.to_csv('zomatov2.csv')

#TF IDF

In [ ]:
from google.colab import drive
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [ ]:
#importing dataset
drive.mount('/content/drive')
csv_file_path = '/content/drive/My Drive/Zomato Recommender/Dataset/zomatov2.csv'
zomato = pd.read_csv(csv_file_path, usecols=lambda column: column != 'Unnamed: 0')
print(zomato.shape)
zomato.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(40126, 10)


,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,Jalsa,True,True,4.1,Banashankari,"North Indian, Mughlai, Chinese",800,rated 40 ratedn beautiful place dine inthe int...,Banashankari,3.99
1,Spice Elephant,True,False,4.1,Banashankari,"Chinese, North Indian, Thai",800,rated 40 ratedn dinner family turned good choo...,Banashankari,3.97
2,San Churro Cafe,True,False,3.8,Banashankari,"Cafe, Mexican, Italian",800,rated 30 ratedn ambience good enough pocket fr...,Banashankari,3.58
3,Addhuri Udupi Bhojana,False,False,3.7,Banashankari,"South Indian, North Indian",300,rated 40 ratedn great food proper karnataka st...,Banashankari,3.45
4,Grand Village,False,False,3.8,Basavanagudi,"North Indian, Rajasthani",600,rated 40 ratedn good restaurant neighbourhood ...,Banashankari,3.58


In [ ]:
# Randomly sample 10% of dataframe
df_percent = zomato.sample(frac=0.1)
df_percent.shape

(4013, 10)

In [ ]:
# Changing the cuisines and city to lists
df_percent['cuisines'] = df_percent['cuisines'].apply(lambda x: x.split(', '))
df_percent['city'] = df_percent['city'].apply(lambda x: [x])

# Display the DataFrame to verify the changes
df_percent.head()

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
31279,Mysore Hanumanthu Palav,True,False,3.9,Rajajinagar,"[South Indian, Fast Food]",500,rated 50 ratedn best biryani town itãx83ãx83...,[Malleshwaram],3.71
23198,Waffle Stories,True,False,4.0,BTM,"[Desserts, Beverages]",300,rated 40 ratedn loved place desserts presentat...,[Koramangala 5th Block],3.73
13746,Mad Over Donuts,True,False,3.9,Brigade Road,"[Bakery, Desserts, Beverages]",450,rated 40 ratedn went late night dinnernnthey g...,[Indiranagar],3.75
34360,Just@99,True,False,3.1,New BEL Road,"[Chinese, South Indian, North Indian]",300,rated 50 ratedn ordered manchurian soup chicke...,[New BEL Road],2.68
31786,Real Fresh Dosa Corner,True,False,4.2,Brookefield,[South Indian],200,rated 40 ratedn ordered masala dosa kerala mas...,[Marathahalli],3.84


In [ ]:
df_percent.set_index('name', inplace=True)

indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])

In [ ]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

###Restaurant Names

In [ ]:
list(df_percent.index.unique())

['Mysore Hanumanthu Palav',
 'Waffle Stories',
 'Mad Over Donuts',
 'Just@99',
 'Real Fresh Dosa Corner',
 'Pulimunchi',
 'New Kabab Zone',
 'Onesta',
 'Levitate Brewery And Kitchen',
 'The Green Path - Forgotten Food',
 'New Friends',
 'Shanghai Times',
 'The Milkshake Theory',
 'Tandoor Merchant',
 'The Kitchen Story',
 'Head O State',
 'Cafe Mondo',
 'Palmgrove',
 'Faasos',
 'Mughals Restaurant',
 'Prems Graama Bhojanam',
 'Karachi Bakery',
 'Guzzlers Inn',
 'Chaithanya Delicacy',
 '1To3 Kitchen',
 'Szechuan Dragon',
 'Monarch',
 'Indiana Burgers',
 "Andy'S Cafã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â\x83Ã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x82Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â©",
 'Parisar Veg Restaurant',
 'Barbeque Nation',
 'Armaani Caffe',
 'Yumyumsouth',
 'Nalukettu',
 'Two Friends Cauldron',
 'Indira Darshini',
 'S L V Food Joint',
 'Gyani Da Punjabi Dhaba',
 'Dabba Karkhana',
 'Monsoon',
 'Rolls On Wheels',
 'Red Onion',
 'Moriz Restauran

###Just checking some restaurant names with filtered cuisines to check if the recommender is recommending good!

In [ ]:
# Using boolean indexing to filter restaurants with "Maharastrian" cuisine
mh_restaurants = df_percent[df_percent['cuisines'].apply(lambda x: 'Maharashtrian' in x)]

# Displaying the filtered DataFrame
mh_restaurants


,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,,,
The Square Table,True,True,4.2,Whitefield,"[North Indian, Maharashtrian, South Indian]",1000,rated 40 ratedn whitefield crave delicious nor...,[Whitefield],4.10
Bombay Talkies,True,False,3.7,Jayanagar,"[Maharashtrian, Fast Food, Street Food, Sandwich]",200,rated 10 ratedn misal much tastyni expected mi...,[JP Nagar],3.43
Victoria Terminus,False,False,3.4,Ejipura,"[Fast Food, Maharashtrian, Rolls]",250,rated 35 ratedn chain quick service restaurant...,[Koramangala 6th Block],2.79
Bombay Talkies,True,False,3.6,Jayanagar,"[Maharashtrian, Fast Food, Street Food, Sandwich]",200,rated 20 ratedn visited place taste mumbai wal...,[Koramangala 7th Block],3.43
Bombay Talkies,True,False,3.7,Jayanagar,"[Maharashtrian, Fast Food, Street Food, Sandwich]",200,rated 10 ratedn misal much tastyni expected mi...,[BTM],3.43
Purnabramha Maharashtrian Restaurant,True,False,3.7,HSR,[Maharashtrian],850,rated 20 ratedn food 15nnambience 45nnservice ...,[HSR],3.35
Malhar Maharashtrian Cuisine,True,False,4.0,Bannerghatta Road,[Maharashtrian],500,rated 40 ratedn one simple sweet place find vi...,[JP Nagar],3.84
Kollapuri'S,False,False,3.6,Jayanagar,[Maharashtrian],600,rated 45 ratedn favourite place timensatisfact...,[Jayanagar],3.52
The Square Table,True,True,4.2,Whitefield,"[North Indian, Maharashtrian, South Indian]",1000,rated 20 ratedn one places ended trying due zo...,[Brookefield],4.10


In [ ]:
# Using boolean indexing to filter restaurants with "Biryani" cuisine
biriyani_restaurants = df_percent[df_percent['cuisines'].apply(lambda x: 'Biryani' in x)]

# Displaying the filtered DataFrame
list(biriyani_restaurants.index)


['Faasos',
 'Mughals Restaurant',
 'Red Onion',
 'Moriz Restaurant',
 'Chinese Chaska',
 'Andhra Ruchulu',
 'Moolehatti Donne Biryani Mane',
 'Gundappa Donne Biryani',
 'Masala Oota',
 '100Ã\x83Â\x83Ã\x82Â\x83Ã\x83Â\x82Ã\x82Â\x82Ã\x83Â\x83Ã\x82Â\x82Ã\x83Â\x82Ã\x82Â°C',
 'Xpress Biryani',
 'Khan Saab Hotel',
 'Hotel Annapoorna',
 'Village Donne Biryani',
 'Donne Biryani Mane',
 'A Southern Fair',
 'Aqni',
 'Bisimillah Biriyani',
 'Hunger2Eat',
 'Abhiruchi Hotel',
 'Ultra Biriyani Paradise',
 'Paradise Premium',
 'Sree Abhiruchi',
 'Halli Donne Biryani Nati Style',
 'Sree Spicy Hub',
 'Nite Out',
 'Moriz Restaurant',
 'Punjabi Tasty Khana',
 'Hyderabad Biryaani Palace',
 'Omg Biriyani',
 'Biryani Miya',
 'Omg Biriyani',
 'Donne Biriyani House',
 'The Biriyani Pedia',
 'Ambur Hot Dum Biriyani',
 'Anand Donne Biriyani',
 "Ammi'S Biryani",
 'Hotel Salala',
 'Royal Treat',
 'Al Hayath Ambur Biryani',
 'Mangalore Pearl',
 'Empire Restaurant',
 'Bawarchi Biryani',
 'Budget Food',
 'Mangalore P

#Recommendation

In [ ]:
def recommend(name, cosine_similarities=cosine_similarities, df_percent=df_percent):
    # Find the index of the restaurant entered
    idx = indices[indices == name].index[0]

    # Details of the chosen restaurant
    #print(df_percent.loc[[name]].head(1))

    # Get chosen restaurant's cuisine and location as strings
    chosen_cuisine = str(df_percent.loc[name, 'cuisines'][0])  # Convert to string
    chosen_city = str(df_percent.loc[name, 'city'][0])  # Convert to string
    print("Chosen restaurant:", name)
    print("Chosen cuisine:", chosen_cuisine)
    print("Chosen city:", chosen_city)

    # Find the restaurants with similar cosine similarity values and order them
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Extract the top 30 similar restaurant indexes
    top30_indexes = [i[0] for i in sim_scores[1:31]]

    # Select the data of top 30 similar restaurants
    df_new = df_percent.iloc[top30_indexes]

    # Remove the input restaurant from the recommended list
    df_new = df_new[df_new.index != name]

    # Explode cuisines and locations into separate rows
    df_new = df_new.explode('cuisines')
    df_new = df_new.explode('city')

    # Filter restaurants with the same cuisine or in the same location
    df_soft = df_new[(df_new['cuisines'] == chosen_cuisine) | (df_new['city'] == chosen_city)]

    # Conditional assignment based on whether df_soft is empty or not
    if df_soft.empty:
        print('\n Sorry! we can not find the same cuisine in the same city. But here is a list of restaurants you may like:\n\n')
        df_new = df_new
    else:
        df_new = df_soft

    # Drop duplicate entries based on the first column
    df_new = df_new.reset_index().drop_duplicates(subset='name', keep='first').set_index('name')


    # Sort by 'rate' column in descending order and select the top 10
    df_new = df_new.sort_values(by='rate', ascending=False).head(10)

    # Select only the columns you want to display
    columns_to_display = ['rate', 'location', 'cuisines', 'cost', 'reviews_list', 'city', 'Mean Rating']
    df_new = df_new[columns_to_display]

    print('Top %s restaurants like %s with similar reviews and cuisine OR city: ' % (str(len(df_new)), name))

    return df_new

#recommend('Kerala Mess Upahar')
recommend("Dadi'S Dum Biryani")
#recommend('Namma Halli Jonne Biryani')
#recommend('Le Jardin - The Oberoi')


Chosen restaurant: Dadi'S Dum Biryani
Chosen cuisine: ['Biryani', 'North Indian', 'Mughlai']
Chosen city: ['JP Nagar']

 Sorry! we can not find the same cuisine in the same city. But here is a list of restaurants you may like:


Top 10 restaurants like Dadi'S Dum Biryani with similar reviews and cuisine OR city: 


,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,
Hammered,4.7,Cunningham Road,North Indian,1300,rated 50 ratedn really good place visit friend...,MG Road,4.65
Flechazo,4.7,Marathahalli,Asian,1400,rated 40 ratedn great place lunch buffetngood ...,Marathahalli,4.87
Farzi Cafe,4.4,Lavelle Road,Modern Indian,1500,rated 40 ratedn amazing food ambience absolute...,Residency Road,4.35
Meghana Foods,4.3,Residency Road,Biryani,600,rated 40 ratedn went colleagues lunchplace cro...,Lavelle Road,4.28
The Black Rabbit,4.3,Indiranagar,Continental,1500,rated 50 ratedn second visit time friday eveni...,Old Airport Road,4.23
Hoot,4.2,Sarjapur Road,Continental,1400,rated 30 ratedn first official outing mine fri...,Sarjapur Road,4.10
Laughing Llama Gastropub,4.2,Brigade Road,Finger Food,1600,rated 40 ratedn wowiekazawinamazing place wish...,MG Road,4.16
K.G.N Kabbab Corner,4.0,Kalyan Nagar,North Indian,500,rated 50 ratedn amazing place iãx83ãx83ãx82...,Kammanahalli,3.84
Mani'S Dum Biryani,4.0,Jeevan Bhima Nagar,Biryani,750,rated 30 ratedn high expectations ordered mutt...,Old Airport Road,3.77


In [ ]:
recommend('Delight Food')

Chosen restaurant: Delight Food
Chosen cuisine: Andhra
Chosen city: Koramangala 7th Block
Top 10 restaurants like Delight Food with similar reviews and cuisine OR city: 


,rate,location,cuisines,cost,reviews_list,city,Mean Rating
name,,,,,,,
Nagarjuna,4.3,Koramangala 5th Block,Andhra,800,rated 40 ratedn go eat chicken biryani andhra ...,BTM,4.20
Meghana Foods Grand,4.2,Marathahalli,Andhra,600,rated 50 ratedn oh love meghnas biriyani never...,Brookefield,4.10
Biryanis And More,4.1,Jayanagar,Andhra,750,rated 50 ratedn dear biryanis moreni cannot ex...,Jayanagar,3.62
Mani'S Dum Biryani,4.0,Bellandur,Andhra,750,rated 40 ratedn tried andhra paneer biryani ha...,Sarjapur Road,3.77
Bheema'S,4.0,Church Street,Andhra,650,rated 40 ratedn much hype created hubz decided...,Church Street,3.84
Andhra Ruchulu,4.0,Old Airport Road,Andhra,400,rated 50 ratedn online order nni keep ordering...,Indiranagar,3.72
Bahar Cafe,4.0,Marathahalli,Andhra,750,rated 30 ratedn lil lazy go outside thought or...,Brookefield,3.84
Spice Kitchen,3.9,Kalyan Nagar,Andhra,750,rated 40 ratedn wat yummilicious hyderabadi mu...,Kammanahalli,3.68
Kritunga Restaurant,3.8,Koramangala 5th Block,Andhra,650,rated 30 ratedn visited multiple time friendsp...,BTM,3.26
